# Improving performance 📈

In this notebook, we'll see how to use the profiler to see what code ran on the GPU and what ran on the CPU. We'll see that rewriting code to keep everything on the GPU results in optimal performance.

In [ ]:
%load_ext cudf.pandas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
pd.DataFrame()  # warmup

Let's generate some data and do some timeseries operations with it:

In [ ]:
%%time

rng = pd.date_range("2023-01-01", "2023-02-01", freq="10ms")
data = pd.DataFrame(
    {
        "a": np.random.rand(len(rng)),
        "b": np.random.rand(len(rng))
    },
    index=rng
)
data = data.iloc[rng.indexer_between_time("09:30", "16:00")]
results = data.groupby(pd.Grouper(freq="1D")).mean()
results.head()

That runs quite slowly, even when `cudf.pandas` is enabled. Notice what happens when you run the same code with the `%%cudf.pandas.profile` magic: 

In [ ]:
%%cudf.pandas.profile

rng = pd.date_range("2023-01-01", "2023-02-01", freq="10ms")
data = pd.DataFrame(
    {
        "a": np.random.rand(len(rng)),
        "b": np.random.rand(len(rng))
    },
    index=rng
)
data = data.iloc[rng.indexer_between_time("09:30", "16:00")]
results = data.groupby(pd.Grouper(freq="1D")).mean()
results.head()

We get a report that shows what functions executed on the GPU, and what functions executed on the CPU. In the code above, everything executed on the GPU except for the `indexer_between_time` function, which is [not supported by the cuDF library](https://docs.rapids.ai/api/cudf/stable/user_guide/api_docs/). 

The key to getting great performance with `cudf.pandas` is to minimize the number of operations that fall back to CPU. While rarely, this cannot be avoided, it often can be achieved with simple rewrites of your code. Let's rewrite the code to use operations that _are_ supported by cuDF:

In [ ]:
%%time

rng = pd.date_range("2023-01-01", "2023-02-01", freq="100ms")
data = pd.DataFrame(
    {
        "a": np.random.rand(len(rng)),
        "b": np.random.rand(len(rng))
    },
    index=rng
)
data = data.iloc[((rng.hour >= 9) & (rng.minute >= 30)) | (rng.hour <= 16)]
results = data.groupby(pd.Grouper(freq="1D")).mean()
results.head()

Not only is this _much_ faster on the GPU, but it's also quite a bit faster on the CPU - a nice win-win!